In [14]:
import requests, json, tqdm, base64
import pandas as pd

In [15]:
#mkdir data

In [16]:
urls = [
    "http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net/2019-03-02/LA_TRANSITION_ECOLOGIQUE.json", 
    "http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net/2019-03-02/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.json",
    "http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net/2019-03-02/DEMOCRATIE_ET_CITOYENNETE.json", 
    "http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net/2019-03-02/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.json", 
    "http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net/2019-03-02/EVENTS.json",
]

#!wget -c https://grandeannotation.fr/download -O actions.zip && unzip actions.zip



In [17]:
def download_answers(url, path_):
    file_name = url.split('/')[-1]
    data = requests.get(url).json()
    with open("{0}/{1}".format(path_, file_name), "w") as f: 
        json.dump(data, f)

In [18]:
[download_answers(url, "./data") for url in tqdm.tqdm(urls)]



  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [00:49<03:17, 49.46s/it]

 40%|████      | 2/5 [01:28<02:18, 46.25s/it]

 60%|██████    | 3/5 [02:42<01:49, 54.56s/it]

 80%|████████  | 4/5 [03:37<00:54, 54.73s/it]

100%|██████████| 5/5 [03:38<00:00, 38.58s/it]

[None, None, None, None, None]

In [25]:
annotations = pd.read_csv("./actions.csv")

In [27]:
def open_answers(path_):
    with open(path_, "r") as f: 
        return json.load(f)

In [30]:
def format_values(json_answers):
    data = []
    for contrib in tqdm.tqdm(json_answers):
        for answer in contrib.get("responses", []):
            tmp_answer = answer
            tmp_answer["reference"]  = contrib["reference"]
            tmp_answer["id"] = contrib["id"]
            data.append(tmp_answer)
            
    return data

In [31]:
transition_eco = format_values(open_answers("./data/LA_TRANSITION_ECOLOGIQUE.json"))
fiscalite = format_values(open_answers("./data/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.json"))
events = format_values(open_answers("./data/EVENTS.json"))
democratie = format_values(open_answers("./data/DEMOCRATIE_ET_CITOYENNETE.json"))
organ = format_values(open_answers("./data/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.json"))



  0%|          | 0/87562 [00:00<?, ?it/s]

  8%|▊         | 6581/87562 [00:00<00:01, 65807.55it/s]

 18%|█▊        | 16063/87562 [00:00<00:00, 72458.19it/s]

 33%|███▎      | 28714/87562 [00:00<00:00, 83109.80it/s]

 48%|████▊     | 42419/87562 [00:00<00:00, 94191.06it/s]

 65%|██████▍   | 56855/87562 [00:00<00:00, 105153.54it/s]

 81%|████████  | 70980/87562 [00:00<00:00, 113884.39it/s]

 97%|█████████▋| 85240/87562 [00:00<00:00, 121205.67it/s]

100%|██████████| 87562/87562 [00:00<00:00, 121382.89it/s]

  0%|          | 0/113521 [00:00<?, ?it/s]

 13%|█▎        | 15223/113521 [00:00<00:00, 152227.97it/s]

 32%|███▏      | 36632/113521 [00:00<00:00, 166676.06it/s]

 54%|█████▎    | 60882/113521 [00:00<00:00, 183928.07it/s]

 75%|███████▌  | 85646/113521 [00:00<00:00, 199310.99it/s]

 99%|█████████▉| 112499/113521 [00:00<00:00, 216014.89it/s]

100%|██████████| 113521/113521 [00:00<00:00, 224701.71it/s]

  0%|          | 0/9501 [00:00<?, ?it/s]

100%|██████████| 9501/9501 [00:00<00:00,

In [36]:
list_of_data = transition_eco + fiscalite + events + democratie + organ

In [38]:
all_contributions = pd.DataFrame(list_of_data) 

In [41]:
clean = all_contributions.dropna(subset=["formattedValue"])

In [51]:
def turn_question_in_id(question_id):
    return base64.b64encode("Question:{0}".format(question_id).encode()).decode()

turn_question_in_id(162)

'UXVlc3Rpb246MTYy'

In [52]:
annotations["question_id"] = annotations.Question.apply(lambda x: turn_question_in_id(x))

In [61]:
clean = clean[clean.questionId.isin(set(annotations.question_id))]

In [70]:
annotations.rename({"Contribution":"reference"}, axis=1, inplace=True)

In [71]:
annotation_with_questions = clean.merge(annotations, 
                                        left_on=["questionId", "reference"],
                                        right_on=["question_id", "reference"], 
                                        how="inner")

In [195]:
annotation_with_questions

,formattedValue,id,questionId,questionTitle,reference,value,Debat,Question,Categorie,Annotateur,Poids,question_id
0,L'Etat,UHJvcG9zYWw6NTk2Nzk5MmItMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTU3,Et qui doit selon vous se charger de vous prop...,2-7,L'Etat,2,157,Etat,13,1.000000,UXVlc3Rpb246MTU3
1,L'Etat,UHJvcG9zYWw6NTk2Nzk5MmItMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTU3,Et qui doit selon vous se charger de vous prop...,2-7,L'Etat,2,157,Etat,247,1.000000,UXVlc3Rpb246MTU3
2,L'Etat,UHJvcG9zYWw6NTk2Nzk5MmItMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTU3,Et qui doit selon vous se charger de vous prop...,2-7,L'Etat,2,157,Etat,269,1.000000,UXVlc3Rpb246MTU3
3,L'Etat,UHJvcG9zYWw6NTk2Nzk5MmItMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTU3,Et qui doit selon vous se charger de vous prop...,2-7,L'Etat,2,157,Etat,1,1.000000,UXVlc3Rpb246MTU3
4,L'Etat,UHJvcG9zYWw6NTk2Nzk5MmItMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTU3,Et qui doit selon vous se charger de vous prop...,2-7,L'Etat,2,157,Etat,17,1.000000,UXVlc3Rpb246MTU3
5,Climat,UHJvcG9zYWw6ZWJlZmM2NjEtMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTQ3,"Si oui, de quelle manière votre vie quotidienn...",2-12,Climat,2,147,"Evénements extrêmes (incendie, inondation, séc...",236,1.787623,UXVlc3Rpb246MTQ3
6,Climat,UHJvcG9zYWw6ZWJlZmM2NjEtMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTQ3,"Si oui, de quelle manière votre vie quotidienn...",2-12,Climat,2,147,"Saisons modifiées, déréglées",297,1.787623,UXVlc3Rpb246MTQ3
7,Climat,UHJvcG9zYWw6ZWJlZmM2NjEtMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTQ3,"Si oui, de quelle manière votre vie quotidienn...",2-12,Climat,2,147,"Saisons modifiées, déréglées",469,1.787623,UXVlc3Rpb246MTQ3
8,Climat,UHJvcG9zYWw6ZWJlZmM2NjEtMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTQ3,"Si oui, de quelle manière votre vie quotidienn...",2-12,Climat,2,147,"Saisons modifiées, déréglées",312,1.787623,UXVlc3Rpb246MTQ3
9,collectivités locales,UHJvcG9zYWw6ZWJlZmM2NjEtMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTU3,Et qui doit selon vous se charger de vous prop...,2-12,collectivités locales,2,157,Collectivités locales,166,1.039830,UXVlc3Rpb246MTU3


In [86]:
df_agg = annotation_with_questions.groupby(["reference", "question_id"]).agg({
    "Categorie": lambda x: set(x.tolist())
}).reset_index()

In [87]:
df_agg.head()

,reference,question_id,Categorie
0,1-100,UXVlc3Rpb246MTA3,"{Z Autres, Société civile, citoyens}"
1,1-10002,UXVlc3Rpb246MTA3,{A Moi-même ou personne}
2,1-10004,UXVlc3Rpb246MTA3,"{X Si valeurs, honnête, Toutes les institution..."
3,1-10004,UXVlc3Rpb246MTEz,{Q Proportionnelle}
4,1-10005,UXVlc3Rpb246MTI3,"{Sans réponse / Hors sujet / Inclassable, Y Rien}"


In [ ]:
annotation_with_questions

In [95]:
df_grouped = df_agg.merge(annotation_with_questions.drop_duplicates(subset=["reference", "question_id"]),
                          left_on=["reference", "question_id"], 
                          right_on=["reference", "questionId"], 
                         how="left")


In [196]:
df_grouped.head()

,reference,question_id_x,Categorie_x,formattedValue,id,questionId,questionTitle,value,Debat,Question,Categorie_y,Annotateur,Poids,question_id_y
0,1-100,UXVlc3Rpb246MTA3,"{Z Autres, Société civile, citoyens}",Le peuple,UHJvcG9zYWw6ZmQ2OTFiNDMtMWUyNi0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTA3,En qui faites-vous le plus confiance pour vous...,Le peuple,1,107,"Société civile, citoyens",2,1.087429,UXVlc3Rpb246MTA3
1,1-10002,UXVlc3Rpb246MTA3,{A Moi-même ou personne},moi même,UHJvcG9zYWw6MWQ2NThkYWItMWYyYS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTA3,En qui faites-vous le plus confiance pour vous...,moi même,1,107,A Moi-même ou personne,2,1.000545,UXVlc3Rpb246MTA3
2,1-10004,UXVlc3Rpb246MTA3,"{X Si valeurs, honnête, Toutes les institution...",Représentation des français,UHJvcG9zYWw6MjRmY2E5ZTEtMWYyYS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTA3,En qui faites-vous le plus confiance pour vous...,Représentation des français,1,107,Sans réponse / Hors sujet / Inclassable,811,19.158053,UXVlc3Rpb246MTA3
3,1-10004,UXVlc3Rpb246MTEz,{Q Proportionnelle},Proportionnelle,UHJvcG9zYWw6MjRmY2E5ZTEtMWYyYS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTEz,Que faudrait-il faire pour mieux représenter l...,Proportionnelle,1,113,Q Proportionnelle,18,1.000000,UXVlc3Rpb246MTEz
4,1-10005,UXVlc3Rpb246MTI3,"{Sans réponse / Hors sujet / Inclassable, Y Rien}",nsp,UHJvcG9zYWw6MjVjNDNhMTEtMWYyYS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTI3,Que proposez-vous pour renforcer les principes...,nsp,1,127,Sans réponse / Hors sujet / Inclassable,406,2.596769,UXVlc3Rpb246MTI3


In [165]:
import ftfy
import io 
import unidecode

In [197]:
for r, row in df_grouped.iterrows():
    print(row)
    break

reference                                                     1-100
question_id_x                                      UXVlc3Rpb246MTA3
Categorie_x                    {Z Autres, Société civile, citoyens}
formattedValue                                            Le peuple
id                UHJvcG9zYWw6ZmQ2OTFiNDMtMWUyNi0xMWU5LTk0ZDItZm...
questionId                                         UXVlc3Rpb246MTA3
questionTitle     En qui faites-vous le plus confiance pour vous...
value                                                     Le peuple
Debat                                                             1
Question                                                        107
Categorie_y                                Société civile, citoyens
Annotateur                                                        2
Poids                                                       1.08743
question_id_y                                      UXVlc3Rpb246MTA3
Name: 0, dtype: object


In [201]:
def clean_tags(tags):
    return [tag if len(tag.split()[0])>1 else " ".join(tag.split()[1:]) for tag in tags]
    

In [202]:
train_data = [{
    "question": row["questionTitle"],
    "answer": row["formattedValue"],
    "target": clean_tags(row.Categorie_x)
} for  r, row  in df_grouped.iterrows()]

In [206]:
with open("data/data_train.json", "w") as f:
    json.dump(train_data, f)

In [204]:
len(train_data)

125693

In [207]:
rm data_train.jsondd

In [169]:
with io.open("data/train.txt", "w", encoding="utf-8") as f:
    for r, row in df_grouped.iterrows():
        categories = " #" + " #".join(["_".join(tag.split()) for tag in row.Categorie_x])
        value = row.formattedValue.encode('utf-8').decode('utf-8') + \
        categories.encode('utf-8').decode('utf-8') + "\n"
        f.write(value)

In [161]:
!head ./data/train.txt

Le peuple #Z_Autres #Soci��t��_civile,_citoyens
moi m��me #A_Moi-m��me_ou_personne
Repr��sentation des fran��ais #X_Si_valeurs,_honn��te #Toutes_les_institutions #X_Si_id��es #Sans_r��ponse_/_Hors_sujet_/_Inclassable #X_Si_actions #Tous_les_��lus #Soci��t��_civile,_citoyens
Proportionnelle #Q_Proportionnelle
nsp #Sans_r��ponse_/_Hors_sujet_/_Inclassable #Y_Rien


In [193]:
[elt for elt in df_grouped.formattedValue.tolist() if ("racailles" in elt)]


['Les abrutis sur la routes et les racailles dans le train!',
 'La sécurité, la protection et la justice, si toutefois cela était correctement pensé. C’est à dire arrêter de donner autant de pouvoir aux médias, faire fermer leur bouche à tous ceux qui mettent des vidéos sur internet sorties du contexte pour alimenter une polémique, arrêter de montrer à la télévision des voleurs, bandits, escrocs, drogués, casseurs, racailles qui se fichent de la police ou de la justice et qui s’en sortent au bout de 48h les bras levés en guise de victoire, arrêter de montrer qu’on met en prison quelqu’un qui va aider une femme ou une personne âgée simplement parce qu’il aura réussi à casser trois dents et un bras au bandit l’ayant agressée et entendre dire par la suite qu’au delà du bandit c’était un homme en difficulté etc etc etc, arrêtez!!!',
 "il y en a tellement qu'on ne sait pas par quoi commencer.Je dirais en priorité l'aide au logement,,,surtout si cela est coordonné avec un encouragement à viv